# Cognitive Stress Detection from Wearable Physiological Signals

**Goal**: Build a classifier to detect cognitive stress vs rest states using physiological data from wearable devices.

## Dataset Overview:
- **22 subjects** (10 males V1 protocol, 12 females V2 protocol)
- **Signals**: EDA, BVP, HR, IBI (for HRV), Temperature, Accelerometer
- **Cognitive Tasks**: Stroop test, TMCT, Opinion speeches, Subtraction test
- **Rest Periods**: Baseline, rest between tasks

## Stress Induction Protocol

### First version (S01 to S18)

![Stress illustration](/Users/camilletran/Documents/Capstone/behavior-cap/Capstone-Behavorial/stress_v1.png)

### Second version (f01 to f18)

![Stress illustration](/Users/camilletran/Documents/Capstone/behavior-cap/Capstone-Behavorial/stress_v2.png)

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os
import datetime
from scipy import stats

In [ ]:
dataset_path = '22subjects/STRESS'
stress_level_v1_path = 'Stress_Level_v1.csv'
stress_level_v2_path = 'Stress_Level_v2.csv'
subject_info_path = 'subject-info.csv'